In [1]:
%load_ext autoreload
%autoreload 2

# Example of a pvsystemprofiler function


## 8/10/2021

### Notebook setup and library imports

In [2]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# PVInsight Code Imports
from solardatatools import DataHandler
from solardatatools.dataio import get_pvdaq_data
sys.path.append('..')
sys.path.append('~/github/pv-system-profiler')
from pvsystemprofiler.estimator import ConfigurationEstimator
from pvsystemprofiler.longitude_study import LongitudeStudy

### Load data table from external source

For today's example, we're loading data from NREL's PVDAQ API, which is a publically available PV generatation data set. 

In [3]:
data_frame = get_pvdaq_data(sysid=1199, year=[2015, 2016, 2017], api_key='DEMO_KEY')

CAUTION: Multiple scan rates detected!=======================] 100.0% ...queries complete in 26.7 seconds       
Scan rates (in seconds): [300, 280]
0 transitions detected.
Suggest splitting data set between:




In [4]:
data_frame.head()

AttributeError: 'tuple' object has no attribute 'head'

### Running the `DataHandler` processing pipeline

In [ ]:
dh = DataHandler(data_frame)

In [ ]:
dh.run_pipeline(power_col='ac_power')

### Site longitude and latitude downloaded from https://maps.nrel.gov/pvdaq/ PVDAQ Contributed Sites

In [ ]:
latitude = 39.4856
longitude = -76.6636
tilt = 180
azimuth = 20
gmt_offset = -5

### Import sunrise_sunset method to calculate the daily solar noon

In [ ]:
from pvsystemprofiler.algorithms.optimized_sunrise_sunset import get_optimized_sunrise_sunset

### Prepare input values to estimate the daily sunrise and sunset hour

### For this function, one of the input values is set to None at one time. It gives the option to calculate sunrise and sunset hours using either the raw data matrix (rdm) or the filled data matrix (fdm)

In [ ]:
#fdm = None
#rdm = dh.raw_data_matrix
fdm = dh.filled_data_matrix
rdm = None

In [ ]:
day_of_year = dh.day_index.dayofyear

In [ ]:
days = dh.daily_flags.no_errors

### The method opt_dict returns a dictionary with the daily sunrise and sunset hours and the threshold value

In [ ]:
opt_dict = get_optimized_sunrise_sunset(fdm, rdm)

In [ ]:
opt_dict

### Estimate solar noon using estimated values in the dictionary

In [ ]:
sunrise = opt_dict['meas_sr_f']
sunset = opt_dict['meas_ss_f']
sunrise[np.isnan(sunrise)] = 0
sunset[np.isnan(sunset)] = 0

In [ ]:
solarnoon = np.nanmean([sunrise, sunset], axis=0)

In [ ]:
from pvsystemprofiler.utilities.equation_of_time import eot_da_rosa, eot_duffie

### Equation of time prosposed by Duffie

In [ ]:
eot_duffie = eot_duffie(day_of_year)

### Equation of time prosposed by da Rosa

In [ ]:
eot_da_rosa = eot_da_rosa(day_of_year)

### Fit longitude using previously estimated solarnoon

In [ ]:
from pvsystemprofiler.algorithms.longitude.fitting import fit_longitude

In [ ]:
fit_longitude(eot_duffie, solarnoon, days, gmt_offset, loss='l2')

### Fit longitude using parameters internal to the LongitudeStudy class

In [ ]:
lon_study = LongitudeStudy(data_handler=dh, gmt_offset=gmt_offset, true_value=longitude)

In [ ]:
lon_study.run(verbose=False, data_matrix='filled', estimator='fit_l2', day_selection_method='all',
              eot_calculation='duffie', solar_noon_method='optimized_measurements')

### use solarnoon calculated internally

In [ ]:
solarnoon2 = lon_study.solarnoon

### Use days calculated internally

In [ ]:
day_of_year2 = lon_study.data_handler.day_index.dayofyear

### Use internal eout Duffie

In [ ]:
eot_duffie2 = lon_study.eot_duffie

In [ ]:
days2 = lon_study.days

In [ ]:
fit_longitude(eot_duffie2, solarnoon2, days2, gmt_offset, loss='l2')

In [ ]:
solarnoon